In [1]:
import sys
sys.path.append('C:/Users/keeli/nov23/repos/Literature')
sys.path.append('C:/Users/keeli/nov23/repos/Literature/scripts')

In [2]:
import pandas as pd
import os
import string
from get_biblatex import GetBiblatex
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
from ast import literal_eval
from collections import defaultdict
from semanticscholar import SemanticScholar, SemanticScholarException

C:\Users\keeli\anaconda3\envs\lit-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_item_to_blacklist(item): # item here is a row from the manually checked csv file
    #Add item to blacklist.csv
    move_to_blacklist = {
        'staff_id': item.get('staff_id', None),
        'staff_name': item.get('staff_id', None),
        'ss_year': item.get('ss_year', None),
        'ss_id': item.get('ss_id', None),
        'title': item.get('ss_title', None),
        'doi': item.get('ss_doi', None),
        'Should be in diag.bib': 'no',
        'Reason': item.get('Blacklist reason', None)
    }

    return move_to_blacklist

In [4]:
def update_blacklist_csv(blacklist_df, blacklist_entries, blacklist_out_file): #blacklist_csv is a df
    # Add all items to blacklist.csv
    blacklist_df = pd.concat([blacklist_df, pd.DataFrame(blacklist_entries)], ignore_index=True)

    # Save blacklist.csv
    blacklist_df.to_csv(blacklist_out_file, index=False)
    return f"{len(blacklist_entries)} items added to blacklist"

In [5]:
# Code to get citations from semantic scholar. If there are multiple ss_ids, we should get the number of citations for each of them and sum the two (or more?) values.
def get_citations(semantic_scholar_ids, sch):
    dict_cits = {}
    ss_ids_not_found = []
    for ss_id in semantic_scholar_ids:
        tries = 8
        i=0
        while i<tries:
            print('trying time', i, ss_id)
            try:
                paper = sch.get_paper(ss_id)
                paper_id = paper['paperId']
                dict_cits[paper_id] = len(paper['citations'])
                print('success getting citations')
                i=tries # we succeeded so max out the tries
            except SemanticScholarException.ObjectNotFoundException as onfe:
                ss_ids_not_found.append(ss_id)
                print('failed cleanly to get citations')
                i=tries # we failed cleanly so max out the tries
            except Exception as e: # some kind of time out error
                print('failed to get citations, trying again')
                i = i+1  # if we still have more tries left then try it again
            
    return dict_cits, ss_ids_not_found

In [6]:
def get_bib_info(diag_bib_file, item): #diag_bib_file is the file read in as a string, item is row from csv
    #Get DOI information

    # if no ss_doi exists
    if len(str(item['ss_doi']))==0 or str(item['ss_doi'])=='nan':
        print('no ss_doi available, I cannot add new bib entry', item['ss_id'])
        return None
    
    # make sure doi is not already in diag.bib
    if item['ss_doi'] in diag_bib_file:

        start_index = diag_bib_file.find(item['ss_doi'])
        end_index = diag_bib_file.find('}', start_index)  # Include the closing brace
        matching_item_str = diag_bib_file[start_index:end_index]

        print('DOI already exists in bib file. Matching item:', matching_item_str)

        if matching_item_str == item['ss_doi']:
            print('doi already exists in bib file, I will not add new bib entry', item['ss_doi'], item['ss_id'])
            return None
        
        else:
            print('similar doi already exists in bib file, but new item will be added for ', item['ss_doi'], item['ss_id'])

    # Get BibLatex information based on DOI if not in the file
    reader = GetBiblatex(doi=item['ss_doi'], diag_bib=diag_bib_file)
    bibtext = reader.get_bib_text()

    # Return the bibtext if it is not 'empty', otherwise return None
    return bibtext if bibtext != 'empty' else None





In [7]:
def add_ss_id_doi_pmid_to_existing_bibkey(diag_bib_raw, item_row):
    ss_id = item_row['ss_id']
    bibkey = item_row['bibkey']
    #Update bibkey with ss_id
    for ind, entry in enumerate(diag_bib_raw):
        if entry.type == 'string':
            continue

        # if we found the relevant key
        if bibkey == entry.key:
            # print('entry matched is ', entry.fields)
            # if there is already something in all_ss_ids
            if 'all_ss_ids' in entry.fields.keys():
                if not entry.fields['all_ss_ids'] == '{' + str(ss_id) + '}': # this should never happen, right? (from Keelin!)
                    previous = literal_eval(entry.fields['all_ss_ids'].strip('{}'))
                    new = ss_id
                    combined = list(set(previous) | set([new]))
                    # update the entry
                    entry.fields['all_ss_ids'] = '{' + str(combined) + '}'
            # if there is no ss_id here yet just add this single one
            else:   
                    entry.fields['all_ss_ids'] = '{' + str(ss_id) + '}'
            print(str(ss_id), 'added to diag_bib_raw')

            ss_doi = str(item_row['ss_doi']).strip()
            if not 'doi' in entry.fields.keys() and len(ss_doi)>0:
                print('will add doi to bibkey', bibkey,  ss_doi)
                entry.fields['doi'] = '{' + ss_doi + '}'
            ss_pmid = item_row['ss_pmid'].strip()
            if not 'pmid' in entry.fields.keys() and len(ss_pmid)>0:
                print('will add pmid to bibkey', bibkey,  ss_pmid)
                entry.fields['pmid'] = '{' + ss_pmid + '}'


            return [diag_bib_raw, 'Success']
        
    # if we haven't returned by now then we failed to update 
    print('failed to add ss_id to diag.bib', str(ss_id), str(bibkey))
    return [diag_bib_raw, 'Fail']

In [8]:
def add_pmid_where_possible(diag_bib_raw, dict_bibkey_pmid):
    # iterate through all items in the diag bib and update them if we have missing information on them
    for ind, entry in enumerate(diag_bib_raw):
        if entry.type == 'string':
            continue

        # if we found the relevant key
        current_bibkey = entry.key
        if current_bibkey in dict_bibkey_pmid.keys():
            if not 'pmid' in entry.fields.keys() and len(dict_bibkey_pmid[current_bibkey].strip())>0:
                print('will add pmid to bibkey', current_bibkey,  dict_bibkey_pmid[current_bibkey].strip())
                entry.fields['pmid'] = '{' + dict_bibkey_pmid[current_bibkey].strip() + '}'

    return diag_bib_raw
                
        
                
            
    

In [9]:
def update_citation_count(diag_bib_raw):
    all_ss_ids_not_found = []
    num_entries = len(diag_bib_raw)

    sch = SemanticScholar(timeout=40)
    sch.timeout=40

    for ind, entry in enumerate(diag_bib_raw):
        # print('checking citations', ind, 'of', num_entries)
        flag=0
        if entry.type == 'string':
            continue
        if 'all_ss_ids' in entry.fields:
            all_ss_ids = []
            ss_ids = entry.fields['all_ss_ids'].translate(str.maketrans('', '', string.punctuation)).split(' ')
            if len(ss_ids) > 1:
                all_ss_ids.extend(ss_ids)
            else:
                all_ss_ids.append(ss_ids[0])
            print('trying with key', entry.key, 'and ss ids', all_ss_ids)
            dict_cits, ss_ids_not_found_this_item = get_citations(all_ss_ids, sch)
            if len(ss_ids_not_found_this_item)>0:
                print('adding items to ss_ids_not_found', ss_ids_not_found_this_item)
                all_ss_ids_not_found.extend(ss_ids_not_found_this_item)
            n_cits = 0
            for key in dict_cits.keys():
                n_cits += dict_cits[key]
            print('n_cits this item is ', n_cits)
            # TODO: is it correct logic to use this field name or should we make a new one?
            if 'gscites' in entry.fields:
                # only update if we are increasing the number of citations!!!
                previous_cits = int(entry.fields['gscites'].strip('{}'))
                if n_cits > previous_cits:
                    print('updating', entry.key, 'from', previous_cits, 'to', n_cits)
                    entry.fields['gscites'] = '{' + str(n_cits) + '}'
                elif (previous_cits > (1.5 * n_cits)) and (previous_cits - n_cits > 10):
                    print('warning: num citations calculated for this bibkey is much lower than previously suggested....', entry.key, previous_cits, n_cits)
                else:
                    print('will not update', entry.key, 'as there is no increase', n_cits, previous_cits)
            else:
                print('adding gscites', entry.key, n_cits)
                entry.fields['gscites'] = '{' + str(n_cits) + '}'
    print('done updating citations')
    return diag_bib_raw, all_ss_ids_not_found

# Load manually checked csv file and bib file

In [10]:
# load manually_checked
manually_checked = pd.read_excel("./script_data/manual_check_20231018.xlsx")
manually_checked['ss_pmid'] = manually_checked['ss_pmid'].fillna('-1')
manually_checked['ss_pmid'] = manually_checked['ss_pmid'].astype(int).astype(str)
manually_checked['ss_pmid'] = manually_checked['ss_pmid'].replace('-1', '')

manually_checked['ss_doi'] = manually_checked['ss_doi'].fillna('')


# load bib file just for reading at this point
#TODO: in the end when this script is routine this should just read the live diag.bib
cwd = os.getcwd()
parent_directory = os.path.dirname(cwd)
diag_bib_path = os.path.join(parent_directory, 'scripts/script_data/diag_ss.bib')
with open(diag_bib_path, 'r', encoding="utf8") as orig_bib_file:
    diag_bib_orig = orig_bib_file.read()


In [11]:
# Iterate through all items in the manually checked csv
blacklist_items = []
items_to_add = ''
items_to_update = []

failed_new_items = []
failed_updated_items = []
failed_to_find_actions = []

dict_new_items_bibkey_pmid = {}


for index, bib_item in manually_checked.iterrows():
    print(f"Working on {index}/{len(manually_checked)}: {bib_item['ss_doi']} (action is {bib_item['action']})")
    # Make sure item is manually checked
    if "," in bib_item['action']:
        print(f"{bib_item['ss_id']} has not been checked yet, make sure only 1 action is mentioned")
        failed_to_find_actions.append(bib_item)
        continue

    # Add new item to diag.bib
    elif "[add new item]" == bib_item['action'].strip():
       
       bib_item_text = get_bib_info(diag_bib_orig, bib_item)

       # @Dre it seems like failure results in the return of the text 'empty' - could we return None instead? 
       if bib_item_text is not None:
           items_to_add += bib_item_text
           # if there is a pmid note it to be added afterwards
           ss_pmid = bib_item['ss_pmid'].strip()
           if len(ss_pmid)>0:
               # bit of a hacky way to get the bibkey of the added item
               bibkey_added = bib_item_text[bib_item_text.index('{')+1:bib_item_text.index(',')]
               dict_new_items_bibkey_pmid[bibkey_added] = ss_pmid
               print('storing bibkey and pmid', bibkey_added, ss_pmid)
       else:
           print('failed to find details for doi, ss_id', bib_item['ss_doi'], bib_item['ss_id'])
           failed_new_items.append(bib_item)
       

    # Add ss_id to already existing doi in diag.bib
    elif "[add ss_id]" in bib_item['action'].strip():
        # just store a list of these items for now and we will update the file at the end
        items_to_update += [bib_item]
        
    # Get items to blacklist
    elif "blacklist" in bib_item['action'].strip():
        blacklist_item = get_item_to_blacklist(bib_item)
        blacklist_items.append(blacklist_item)

    # Get None items
    elif '[None]' in bib_item['action'].strip():
        continue
        
    else:
        print('failed to find action', bib_item['action'])
        failed_to_find_actions.append(bib_item)

Working on 0/280: 10.1016/j.media.2022.102605 (action is [add new item])
storing bibkey and pmid Roth22 36156419
Working on 1/280: 10.1177/0271678X18756218 (action is [add new item])
storing bibkey and pmid Lark18 29498562
Working on 2/280: 10.1016/j.ejso.2022.11.378 (action is [add new item])
Working on 3/280:  (action is [blacklist ss_id])
Working on 4/280: 10.1038/s41585-020-0324-x (action is [add new item])
storing bibkey and pmid Omar20 32461687
Working on 5/280:  (action is [add ss_id])
Working on 6/280: 10.1148/radiol.2019182666 (action is [add new item])
storing bibkey and pmid Grob19a 31084482
Working on 7/280:  (action is [blacklist])
Working on 8/280: 10.1016/j.ejso.2020.04.038 (action is [add new item])
storing bibkey and pmid Sand20a 32536526
Working on 9/280: 10.23698/AIDA/BRLN (action is [blacklist])
Working on 10/280: 10.1093/neuonc/now292.004 (action is [blacklist ss_id])
Working on 11/280: 10.1093/cid/ciac623 (action is [add new item])
storing bibkey and pmid Maer22 3

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Working on 83/280: 10.1038/s41523-021-00378-7 (action is [add new item])
Working on 84/280: 10.1158/2326-6066.CIR-20-0741 (action is [add new item])
storing bibkey and pmid Slaa21 34226201
Working on 85/280: 10.1109/CBMS.2017.29 (action is [add new item])
Working on 86/280: 10.1055/s-0040-1713119 (action is [add new item])
storing bibkey and pmid Heba20 32987429
Working on 87/280: 10.1136/ijgc-2022-esgo.790 (action is [add new item])
Working on 88/280: 10.1148/radiol.2017162894 (action is [add new item])
storing bibkey and pmid Bank17 28650738
Working on 89/280:  (action is [add ss_id])
Working on 90/280: 10.1109/TPAMI.2019.2936841 (action is [add ss_id])
Working on 91/280: 10.1117/12.2317937 (action is [add new item])
Working on 92/280: 10.5220/0011669000003414 (action is [add new item])
Working on 93/280: 10.3390/cancers14133260 (action is [add new item])
storing bibkey and pmid Berg22 35805032
Working on 94/280:  (action is [blacklist ss_id])
Working on 95/280: 10.1007/978-3-319-675

storing bibkey and pmid Laur22 37780685
Working on 179/280: 10.48550/arXiv.2006.06356 (action is [add new item])
Working on 180/280: 10.5281/ZENODO.3715652 (action is [blacklist])
Working on 181/280:  (action is [add ss_id])
Working on 182/280:  (action is [None])
Working on 183/280: 10.1007/978-3-319-68548-9_27 (action is [add new item])
Working on 184/280: 10.2214/AJR.18.20786 (action is [add new item])
storing bibkey and pmid Grob19a 30860897
Working on 185/280: 10.1117/12.2254160 (action is [add new item])
Working on 186/280: 10.1007/s00330-021-08035-0 (action is [add new item])
storing bibkey and pmid Kemp21 34019128
Working on 187/280: 10.48550/arXiv.2306.10484 (action is [add new item])
Working on 188/280: 10.1016/j.chest.2020.04.003 (action is [add new item])
storing bibkey and pmid Rubi20 32255413
Working on 189/280: 10.1038/s41585-023-00748-9 (action is [add new item])
storing bibkey and pmid Omar23 37012441
Working on 190/280:  (action is [add ss_id])
Working on 191/280: 10.

Working on 278/280: 10.21037/TLCR-20-924 (action is [add new item])
storing bibkey and pmid Snoe21 34164283
Working on 279/280: 10.1117/12.2318069 (action is [add new item])


# Add new bib entries to the diag.bib file 

In [12]:
# First we use the bib file string, add the completely new bib entries and save it
# append the new items to the string
diag_bib_added_items = diag_bib_orig + items_to_add  
# save the file to disk 
# TODO : write to correct location
diag_bib_path_tmp_new = os.path.join(parent_directory, 'scripts/script_data/diag_ss_tmp_new.bib')
with open(diag_bib_path_tmp_new, 'w', encoding="utf8") as bibtex_file:
    bibtex_file.write(diag_bib_added_items)

# Update newly added items with pmids where possible

In [13]:
# Next we re-open the bib file using the read_bibfile method and update newly added items with new pmids
# TODO read from correct location here
diag_bib_raw = read_bibfile(None, diag_bib_path_tmp_new)
diag_bib_raw = add_pmid_where_possible(diag_bib_raw, dict_new_items_bibkey_pmid)

will add pmid to bibkey Roth22 34100010
will add pmid to bibkey Lark18 29498562
will add pmid to bibkey Omar20 32461687
will add pmid to bibkey Grob19a 30860897
will add pmid to bibkey Sand20a 32737712
will add pmid to bibkey Maer22 35906831
will add pmid to bibkey Wand18 29720220
will add pmid to bibkey Mile17 28622412
will add pmid to bibkey Scha23 37432088
will add pmid to bibkey Harl21 34357130
will add pmid to bibkey Coli21 34014262
will add pmid to bibkey Than22 35351606
will add pmid to bibkey Wand22 35133194
will add pmid to bibkey Berb23 36603288
will add pmid to bibkey Youn21 34388625
will add pmid to bibkey Peis22 35740659
will add pmid to bibkey Toga23 36789976
will add pmid to bibkey Gola21 34425600
will add pmid to bibkey Leij18 29724890
will add pmid to bibkey Li23 37716198
will add pmid to bibkey Reis15 24819231
will add pmid to bibkey Pfaf22 36328273
will add pmid to bibkey Ven15 25979040
will add pmid to bibkey Suzu17 28321915
will add pmid to bibkey Gray17 28964442
w

# Update existing bib entries with new ss_ids (and dois, pmids where possible)

In [14]:
# Next we go over existing items that need ss_ids added (and possibly doi, pmid at same time)
for item_to_update in items_to_update:
    [diag_bib_raw, result] = add_ss_id_doi_pmid_to_existing_bibkey(diag_bib_raw, item_to_update)
    if(result=='Fail'):
        failed_updated_items.append(item_to_update)


#Note we can remove this write - it is just for safety /debug right now
save_to_file(diag_bib_raw, None, diag_bib_path_tmp_new)

03ef312b3d3e616fd7f0a2f2260c82ad62ed7ef1 added to diag_bib_raw
14198a817d2c0a800bfb0a0a36baf5097fe22054 added to diag_bib_raw
14fcdfdd2b15f6fec9b9e7b1b4189e43281273d8 added to diag_bib_raw
1d2109f8ec43c23db647c4778a5bb5846074e575 added to diag_bib_raw
1d3c69edf9e573412de0c758b3db1b8f9996f2c1 added to diag_bib_raw
1dddfd64c4a40269d63014b21ed3ed436f38b98b added to diag_bib_raw
2125835bf1c4fd0646b5dd50855d647044c07658 added to diag_bib_raw
24b8cee45431f633d2fa6e3c05670f62b1e41e7e added to diag_bib_raw
2723ce1686eea776df179e362cd9a8b8e2bb7ff1 added to diag_bib_raw
2a7dafe1287670068300ff77401923f7e151b9f4 added to diag_bib_raw
2e571724830cb8ca6e8dbe9cc1f92fdcfc517ec4 added to diag_bib_raw
2fe9af8a6b41fc9db41e76621f037aac453ab433 added to diag_bib_raw
34559bb0d95c5166625945eef9b53b21a30838fa added to diag_bib_raw
349ca29f588b9c785085da7147a4b58df032a8bf added to diag_bib_raw
374f4a7676183c95f901e655f2caf170cdd9ec9d added to diag_bib_raw
37e383517c34818ad049af0aa763ad5906e9f51a added to diag_

# Update citation counts

In [15]:
ss_ids_not_found_for_citations =[]

In [16]:

diag_bib_raw_new_cits, ss_ids_not_found_for_citations = update_citation_count(diag_bib_raw)

trying with key Abra08a and ss ids ['0445a1a0aa30c9ca41a8ab31463d617b849a0814']
trying time 0 0445a1a0aa30c9ca41a8ab31463d617b849a0814
success getting citations
n_cits this item is  24
trying with key Adam22a and ss ids ['516b2dc6c3761458c8fa6f5759295673e86a42db']
trying time 0 516b2dc6c3761458c8fa6f5759295673e86a42db
success getting citations
n_cits this item is  1
adding gscites Adam22a 1
trying with key Adam22b and ss ids ['c661857719657aac571718e3d48cb66b8fc1e941']
trying time 0 c661857719657aac571718e3d48cb66b8fc1e941
success getting citations
n_cits this item is  11
adding gscites Adam22b 11
trying with key Adri10 and ss ids ['3da4ab4d50211eb7a0148c4ed74729b2d4e30ba9']
trying time 0 3da4ab4d50211eb7a0148c4ed74729b2d4e30ba9
success getting citations
n_cits this item is  7
updating Adri10 from 5 to 7
trying with key Adri11 and ss ids ['691c90e68eab1dcae5861fba0eab73ba626e6777']
trying time 0 691c90e68eab1dcae5861fba0eab73ba626e6777
success getting citations
n_cits this item is  83


success getting citations
trying time 0 94962ef85aca6df5c8289874b56bcaa8722dc596
success getting citations
n_cits this item is  18
updating Balk19 from 8 to 18
trying with key Balk19b and ss ids ['2125835bf1c4fd0646b5dd50855d647044c07658', '66f70286594b27928fb0eea3b4a0f292261c292f']
trying time 0 2125835bf1c4fd0646b5dd50855d647044c07658
success getting citations
trying time 0 66f70286594b27928fb0eea3b4a0f292261c292f
success getting citations
n_cits this item is  65
updating Balk19b from 7 to 65
trying with key Balk20 and ss ids ['2c00230262e1e99862ab27743b725e222a74dcca']
trying time 0 2c00230262e1e99862ab27743b725e222a74dcca
success getting citations
n_cits this item is  20
adding gscites Balk20 20
trying with key Balk21 and ss ids ['25890a20503e79f773f427c999fcb41387f0aab1']
trying time 0 25890a20503e79f773f427c999fcb41387f0aab1
success getting citations
n_cits this item is  17
adding gscites Balk21 17
trying with key Balo18 and ss ids ['3a3664f9b7e6c4258d9dc282c9c43a95054c648e']
try

success getting citations
trying time 0 c3e98e4b01c27affe1b6c161172c5e27e8141618
success getting citations
n_cits this item is  28
updating Bokh19b from 2 to 28
trying with key Bokh23 and ss ids ['f7c9d9a217387d3a73a39b232a3bc14e6706b533']
trying time 0 f7c9d9a217387d3a73a39b232a3bc14e6706b533
success getting citations
n_cits this item is  1
adding gscites Bokh23 1
trying with key Bokh23a and ss ids ['be741d6f455a941f206de7fecb44f81678f385bf']
trying time 0 be741d6f455a941f206de7fecb44f81678f385bf
success getting citations
n_cits this item is  2
adding gscites Bokh23a 2
trying with key Bokh23b and ss ids ['0e7ef01ded474485f8f715342f18907f451526c5']
trying time 0 0e7ef01ded474485f8f715342f18907f451526c5
success getting citations
n_cits this item is  0
adding gscites Bokh23b 0
trying with key Boo09 and ss ids ['40cc7c929546e4422263fa844a8cd6f47d266ead']
trying time 0 40cc7c929546e4422263fa844a8cd6f47d266ead
success getting citations
n_cits this item is  10
trying with key Boo11 and ss id

success getting citations
n_cits this item is  55
adding gscites Brun11 55
trying with key Brun12 and ss ids ['3d346a616566ceafe3a09acdd4831df76c408b33']
trying time 0 3d346a616566ceafe3a09acdd4831df76c408b33
success getting citations
n_cits this item is  135
updating Brun12 from 97 to 135
trying with key Bult18 and ss ids ['add18010e1af63998bae7573f4cd5d2843eeb5bb']
trying time 0 add18010e1af63998bae7573f4cd5d2843eeb5bb
success getting citations
n_cits this item is  16
updating Bult18 from 12 to 16
trying with key Bult18a and ss ids ['eed96a595b9c988affacac34c57128bd115020f0']
trying time 0 eed96a595b9c988affacac34c57128bd115020f0
success getting citations
n_cits this item is  24
updating Bult18a from 8 to 24
trying with key Bult19 and ss ids ['d5b8fba71671814445de80d36a2d563f43eb9d07']
trying time 0 d5b8fba71671814445de80d36a2d563f43eb9d07
success getting citations
n_cits this item is  120
updating Bult19 from 39 to 120
trying with key Bult19a and ss ids ['8ce6b544554a79e077e5fc52f55

success getting citations
n_cits this item is  172
updating Ciom17 from 75 to 172
trying with key Ciom17a and ss ids ['86b9d39fb026746752f95f87f87fd26b8512c913', 'fd0901b1f2121506391a7859de8fb695d159a393']
trying time 0 86b9d39fb026746752f95f87f87fd26b8512c913
success getting citations
trying time 0 fd0901b1f2121506391a7859de8fb695d159a393
success getting citations
n_cits this item is  307
updating Ciom17a from 171 to 307
trying with key Ciom21 and ss ids ['f8208405f959304ba68f3d11d7f880c305ac7b39']
trying time 0 f8208405f959304ba68f3d11d7f880c305ac7b39
success getting citations
n_cits this item is  0
adding gscites Ciom21 0
trying with key Cohe16 and ss ids ['2639e166ecb1dc63dcfe61eefe66b3e2213f17f6']
trying time 0 2639e166ecb1dc63dcfe61eefe66b3e2213f17f6
success getting citations
n_cits this item is  40
updating Cohe16 from 31 to 40
trying with key Cohe16a and ss ids ['670931e779bb4fe0dbb9a9bf3c00e5a53ce0f26b']
trying time 0 670931e779bb4fe0dbb9a9bf3c00e5a53ce0f26b
success getting ci

...

success getting citations
n_cits this item is  0
adding gscites Schr20b 0
trying with key Schr20c and ss ids ['302fbaac065fe76b0f8d6e2bccc8c715d1ab230a']
trying time 0 302fbaac065fe76b0f8d6e2bccc8c715d1ab230a
success getting citations
n_cits this item is  7
adding gscites Schr20c 7
trying with key Schr21 and ss ids ['c4422edc3d7460dc858db3e23cef64136a49628e']
trying time 0 c4422edc3d7460dc858db3e23cef64136a49628e
success getting citations
n_cits this item is  6
adding gscites Schr21 6
trying with key Schr21b and ss ids ['e6826552a5bff552a12087552097519a614ce0c3']
trying time 0 e6826552a5bff552a12087552097519a614ce0c3
success getting citations
n_cits this item is  1
adding gscites Schr21b 1
trying with key Schr21c and ss ids ['53e10b43522e45781bfd4763b49aebba6602c0ac']
trying time 0 53e10b43522e45781bfd4763b49aebba6602c0ac
success getting citations
n_cits this item is  1
adding gscites Schr21c 1
trying with key Schr22 and ss ids ['956fe0cd25879aa006d9885eba32da7100b6d338']
trying time 0

success getting citations
n_cits this item is  19
adding gscites Soly15 19
trying with key Srik12 and ss ids ['e5f7b5efa305a5936b7a89db4a30c35a8e51fe76']
trying time 0 e5f7b5efa305a5936b7a89db4a30c35a8e51fe76
success getting citations
n_cits this item is  2
will not update Srik12 as there is no increase 2 2
trying with key Staa02 and ss ids ['d5ea31e555f04e668dce705c220241750de6b4f6']
trying time 0 d5ea31e555f04e668dce705c220241750de6b4f6
success getting citations
n_cits this item is  14
updating Staa02 from 12 to 14
trying with key Staa04a and ss ids ['ac9748ea3945eb970cc32a37db7cfdfd0f22e74c']
trying time 0 ac9748ea3945eb970cc32a37db7cfdfd0f22e74c
success getting citations
n_cits this item is  1000
trying with key Staa04b and ss ids ['e2a49c9e56d53f9b09ed50de495452bbf74adc03']
trying time 0 e2a49c9e56d53f9b09ed50de495452bbf74adc03
success getting citations
n_cits this item is  9
will not update Staa04b as there is no increase 9 16
trying with key Staa07 and ss ids ['6c776183e47ff076b

success getting citations
trying time 0 d6e975989345b69f539dbb8f22cb3437f5cc5039
success getting citations
n_cits this item is  350
updating Tell19a from 28 to 350
trying with key Tell20 and ss ids ['6441d36a0af25f29abc3a21a012daf1ea9d28594']
trying time 0 6441d36a0af25f29abc3a21a012daf1ea9d28594
success getting citations
n_cits this item is  18
adding gscites Tell20 18
trying with key Tell21 and ss ids ['fed7c0d369d8991f6b76a661dedd0d1768a402d2', '44454c9090606d0332272ff38df6c87eac15f5f7']
trying time 0 fed7c0d369d8991f6b76a661dedd0d1768a402d2
success getting citations
trying time 0 44454c9090606d0332272ff38df6c87eac15f5f7
success getting citations
n_cits this item is  154
adding gscites Tell21 154
trying with key Teuw18 and ss ids ['da42a50fd33cb07f6419f0cd1004b2eda9bdc06a']
trying time 0 da42a50fd33cb07f6419f0cd1004b2eda9bdc06a
success getting citations
n_cits this item is  0
adding gscites Teuw18 0
trying with key Thag23 and ss ids ['13a37cf9771c8fbabc261e13c8a246346244f4ca']
tryin

success getting citations
trying time 0 acaba0aa6a8a55104ae2cb09f093aa30621ffb47
success getting citations
trying time 0 974e8ae2bfa594ba157da2794a3254d12eb7bf26
success getting citations
n_cits this item is  54
adding gscites Vend18 54
trying with key Venh15a and ss ids ['a11db962a303ac6cb6ff4216659189c2ed378c21']
trying time 0 a11db962a303ac6cb6ff4216659189c2ed378c21
success getting citations
n_cits this item is  0
adding gscites Venh15a 0
trying with key Venh15b and ss ids ['ae6efa478f3ecc99f2273effad86f7a8a78a333a']
trying time 0 ae6efa478f3ecc99f2273effad86f7a8a78a333a
success getting citations
n_cits this item is  72
updating Venh15b from 45 to 72
trying with key Venh15c and ss ids ['1d3c69edf9e573412de0c758b3db1b8f9996f2c1']
trying time 0 1d3c69edf9e573412de0c758b3db1b8f9996f2c1
success getting citations
n_cits this item is  13
adding gscites Venh15c 13
trying with key Venh16a and ss ids ['a1b328c04b54decb972bf83458f9df4ab5608af3']
trying time 0 a1b328c04b54decb972bf83458f9df4ab

success getting citations
n_cits this item is  0
adding gscites Vug18 0
trying with key Vuka10 and ss ids ['4ea221e633c584c7abce455e17071d4d95b5fa9c']
trying time 0 4ea221e633c584c7abce455e17071d4d95b5fa9c
success getting citations
n_cits this item is  50
will not update Vuka10 as there is no increase 50 51
trying with key Vuka11 and ss ids ['c072c0566f86cc9bbd967741584f0b5f2d70f0d4']
trying time 0 c072c0566f86cc9bbd967741584f0b5f2d70f0d4
success getting citations
n_cits this item is  7
updating Vuka11 from 4 to 7
trying with key Vuka12 and ss ids ['3e95abdc1bec8622e45fc2258e27f31820754c7b']
trying time 0 3e95abdc1bec8622e45fc2258e27f31820754c7b
success getting citations
n_cits this item is  32
updating Vuka12 from 24 to 32
trying with key Waal15 and ss ids ['1ed7b8605b403640ecb3443ac8097d8cabf60fe0']
trying time 0 1ed7b8605b403640ecb3443ac8097d8cabf60fe0
success getting citations
n_cits this item is  17
updating Waal15 from 13 to 17
trying with key Wand17 and ss ids ['f450e015384d3f06

In [17]:
# TODO: update to the correct output path
save_to_file(diag_bib_raw_new_cits, None, diag_bib_path_tmp_new)

# Update the blacklist

In [18]:
# Last we update the blacklist (temporarily commented) (what failures can happen here?)
blacklist_df = pd.read_csv('./script_data/blacklist.csv')
# TODO: fix to correct output location
blacklist_out_file = './script_data/blacklist_tmp_updated.csv'
# file writing
update_blacklist_csv(blacklist_df, blacklist_items, blacklist_out_file)

'42 items added to blacklist'

In [19]:
# TODO: Here we provide a report of rows where we did not know what to do or we failed to do the action
print("DONE with processing manually checked items")
print('Failures are as follows:')
for item in failed_new_items:
    print('Failed to add new bib entry ', item['ss_id'])
for item in failed_updated_items:
    print('Failed to update exiting bib entry with new ss_id', item['bibkey'], item['ss_id'])
for item in failed_to_find_actions:
    print('Failed to find valid action for item', item['ss_id'], item['action'])
for item in ss_ids_not_found_for_citations:
    print('Failed to find this ss_id to update citations', item)

DONE with processing manually checked items
Failures are as follows:
Failed to add new bib entry  36fb5c86a92b941f0754cb864e2c4c70b21b7b7d
Failed to add new bib entry  e4729ac7bfdb707e3207b0a91b57a2f907f5351b
Failed to find valid action for item 952bdfe8a2732d537363028114718edad19bc451 [add manually]
Failed to find this ss_id to update citations 0f779d6ca748e8a73a78f2e1dadaba3161e80950


In [20]:
print(f"Blacklisted items: {len(blacklist_items)}")
print(f"Updated items: {len(items_to_update)}")
print(f"Newly added items: {items_to_add.count('{yes}')}")
import numpy as np
count_action_none = np.sum(np.fromiter(('none' in str(action).lower() for action in manually_checked['action']), dtype=bool))
print(f"Items with action None: {count_action_none}")


print(f"total processed items: {len(blacklist_items) + len(items_to_update) + items_to_add.count('{yes}') + len(failed_new_items) + len(failed_updated_items) + len(failed_to_find_actions) + count_action_none}")
print(f"amount of items in manual checkfile: {manually_checked.shape[0]}")


Blacklisted items: 42
Updated items: 48
Newly added items: 183
Items with action None: 4
total processed items: 280
amount of items in manual checkfile: 280


In [21]:
# TODO: update to the correct output path
save_to_file(diag_bib_raw_new_cits, None, diag_bib_path_tmp_new)

# Update the blacklist

In [22]:
# Last we update the blacklist (temporarily commented) (what failures can happen here?)
blacklist_df = pd.read_csv('./script_data/blacklist.csv')
# TODO: fix to correct output location
blacklist_out_file = './script_data/blacklist_tmp_updated.csv'
# file writing
update_blacklist_csv(blacklist_df, blacklist_items, blacklist_out_file)

'42 items added to blacklist'

In [23]:
count = items_to_add.count('{yes}')
print(f"Newly added items: {count}")

Newly added items: 183


In [24]:
print(f"Blacklisted items: {len(blacklist_items)}")

Blacklisted items: 42
